In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import dlib
import numpy as np
import Modules.photo_processing as pp
import cv2
import torch
from torch.autograd import Variable
import net_sphere
import sys
import glob
import json
from pathlib import Path


In [ ]:
args = {"--net": 'sphere20a',
        
        "--model": 'drive/My Drive/for_sphereface/sphere20a.pth'}

face_template = np.load('drive/My Drive/for_sphereface/face_template.npy')
predictor = dlib.shape_predictor('drive/My Drive/for_sphereface/shape_predictor_68_face_landmarks.dat')

In [ ]:
INNER_EYES_AND_BOTTOM_LIP = [38, 44, 30, 49, 54]

def embeddings_evaluation(img, landmarks):    

 img_part = pp.alignment(img, landmarks[INNER_EYES_AND_BOTTOM_LIP])
      
 imglist = [img_part,cv2.flip(img_part,1)]
 for i in range(len(imglist)):
        imglist[i] = imglist[i].transpose(2, 0, 1).reshape((1,3,112,96))
        imglist[i] = (imglist[i]-127.5)/128.0
 img_part = np.vstack(imglist)
 with torch.no_grad():
       img_part = Variable(torch.from_numpy(img_part).float()).cuda()
       #img_part = Variable(torch.from_numpy(img_part).float())
 output = net(img_part)
      
 f =  output.cpu().data.numpy()
 #f =  output.data.numpy()
 emb = np.float64(f[0])
 #emb = f
   
 return emb

In [ ]:
net = getattr(net_sphere,args['--net'])()
net.load_state_dict(torch.load(args['--model']))
net.cuda()
net.eval()
net.feature = True


dataset_path = Path('drive/My Drive/Datasets/lfw-deepfunneled')    
    
    
res = {}

for person in dataset_path.glob('*'):
  for img_path in person.glob('*.*'):
    name = img_path.stem
    print(name)
    img = cv2.imread(str(img_path))
    try:
      face_rect = pp.face_detection(img)
      landmarks = pp.landmarks_extracting(img, face_rect[0])
      embs = embeddings_evaluation(img, landmarks)
      res[name] = {
            'landmarks': landmarks.tolist(), 
            'embs': list(embs)
        }
    except:
       print("Error!")   
    


with open('res.txt', 'w') as json_file:
      json.dump(res, json_file)